In [1]:

import pytorch_lightning as pl
import pytorch_lightning.callbacks as pl_callbacks
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import eq


from eq.data import Catalog, InMemoryDataset, Sequence, default_catalogs_dir

In [2]:
anss = eq.catalogs.ANSS_MultiCatalog(    
    num_sequences=100,
    t_end_days=1*365,
    mag_completeness=4.5,
    minimum_mainshock_mag=6.0,
    include_depth=True
)

Generating the catalog...
Downloading...
train


100%|██████████| 60/60 [00:00<00:00, 287.66it/s]


val


100%|██████████| 20/20 [00:00<00:00, 400.67it/s]


test


100%|██████████| 20/20 [00:00<00:00, 421.88it/s]


Success!
Catalog saved to /home/gcl/RA/jonahk/recast/data/ANSS_MultiCatalog


In [3]:
anss.train[0].extra_feat 

tensor([[200000.],
        [ 13400.],
        [343600.],
        [ 32100.],
        [ 34000.],
        [ 31200.],
        [ 27000.],
        [ 75100.],
        [ 41000.],
        [ 10800.],
        [ 44200.],
        [ 43300.],
        [ 41000.],
        [ 41000.],
        [ 38700.],
        [ 33700.],
        [ 37400.],
        [ 43500.],
        [ 41000.],
        [ 22900.],
        [ 44200.],
        [ 27000.],
        [ 45500.],
        [200000.],
        [ 31000.],
        [157800.],
        [525100.],
        [ 33800.],
        [ 35000.],
        [ 35000.],
        [ 35000.],
        [ 84900.]])

In [4]:
dl_train = anss.train.get_dataloader( batch_size=50)
dl_val = anss.val.get_dataloader(batch_size=50)
constant_test = anss.test.get_dataloader(batch_size=50)

In [5]:
model = eq.models.RecurrentTPP(
    num_extra_features=1, # the number of extra features
    tau_mean=(
        len(anss.train[0])/(anss.train[0].t_end-anss.train[0].t_start)
    ), # the mean of the inter-event time distribution
)

In [6]:
    # ModelCheckpoints saves the model with the best validation loss
checkpoint = pl_callbacks.ModelCheckpoint(monitor="total_val_loss")

    # EarlyStopping stops training if the validation loss doesn't improve by more than 1e-3 for 20 epochs
early_stopping = pl_callbacks.EarlyStopping(monitor="total_val_loss", patience=50, min_delta=1e-5)

    # RichProgressBar adds a nice and more functional progress bar
progress_bar = pl_callbacks.RichProgressBar()

    # Trainer set up training and validation loops with previous specs
trainer = pl.Trainer(devices=1,max_epochs=10000, callbacks=[checkpoint, early_stopping],log_every_n_steps=1)

trainer.fit(model, dl_train, dl_val)
checkpoint.best_model_path
model.load_from_checkpoint(checkpoint.best_model_path)
trainer.test(model, constant_test)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/gcl/RA/jonahk/miniconda3/envs/DS-discovery/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name          | Type    | Params
------------------------------------------
0 | hypernet_time | Linear  | 3.2 K 
1 | hypernet_mag  | Linear  | 33    
2 | rnn           | GRU     | 3.6 K 
3 | dropout  

Sanity Checking: 0it [00:00, ?it/s]

/home/gcl/RA/jonahk/miniconda3/envs/DS-discovery/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/gcl/RA/jonahk/miniconda3/envs/DS-discovery/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
/home/gcl/RA/jonahk/miniconda3/envs/DS-discovery/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │     0.618836522102356     │
│      total_test_loss      │     0.618836522102356     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.618836522102356, 'total_test_loss': 0.618836522102356}]

In [ ]:
trainer.test(model, dl_train)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
/home/gcl/RA/jonahk/miniconda3/envs/DS-discovery/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.24940429627895355    │
│      total_test_loss      │    0.24940429627895355    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.24940429627895355, 'total_test_loss': 0.24940429627895355}]

In [ ]:
anss_no_depth = eq.catalogs.ANSS_MultiCatalog(    
    num_sequences=100,
    t_end_days=1*365,
    mag_completeness=4.5,
    minimum_mainshock_mag=6.0,
    include_depth=False
)

Generating the catalog...
Downloading...
train


  0%|          | 0/60 [00:00<?, ?it/s]/home/gcl/RA/jonahk/recast/eq/catalogs/anss.py:153: RuntimeWarning: invalid value encountered in divide
  depth = (depth - np.mean(depth)) / np.std(depth)
/home/gcl/RA/jonahk/recast/eq/catalogs/anss.py:153: RuntimeWarning: divide by zero encountered in divide
  depth = (depth - np.mean(depth)) / np.std(depth)
/home/gcl/RA/jonahk/recast/eq/catalogs/anss.py:153: RuntimeWarning: invalid value encountered in divide
  depth = (depth - np.mean(depth)) / np.std(depth)
100%|██████████| 60/60 [00:00<00:00, 317.33it/s]


val


100%|██████████| 20/20 [00:00<00:00, 366.27it/s]


test


100%|██████████| 20/20 [00:00<00:00, 394.56it/s]


Success!
Catalog saved to /home/gcl/RA/jonahk/recast/data/ANSS_MultiCatalog


In [14]:
no_depth_train = anss_no_depth.train.get_dataloader( batch_size=len(anss_no_depth.train), shuffle=False)
no_depth_val = anss_no_depth.val.get_dataloader(batch_size=len(anss_no_depth.val), shuffle=False)


In [15]:

model = eq.models.RecurrentTPP(
 # the number of extra features
    tau_mean=(
        len(anss_no_depth.train[0])/(anss_no_depth.train[0].t_end-anss_no_depth.train[0].t_start)
    ), # the mean of the inter-event time distribution
)

In [16]:
    # ModelCheckpoints saves the model with the best validation loss
checkpoint = pl_callbacks.ModelCheckpoint(monitor="total_val_loss")

    # EarlyStopping stops training if the validation loss doesn't improve by more than 1e-3 for 20 epochs
early_stopping = pl_callbacks.EarlyStopping(monitor="total_val_loss", patience=50, min_delta=1e-5)

    # RichProgressBar adds a nice and more functional progress bar
progress_bar = pl_callbacks.RichProgressBar()

    # Trainer set up training and validation loops with previous specs
trainer = pl.Trainer(devices=1,max_epochs=10000, callbacks=[checkpoint, early_stopping],log_every_n_steps=1)

trainer.fit(model, no_depth_train, no_depth_val)
checkpoint.best_model_path
model.load_from_checkpoint(checkpoint.best_model_path)
trainer.test(model, constant_test)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name          | Type    | Params
------------------------------------------
0 | hypernet_time | Linear  | 3.2 K 
1 | hypernet_mag  | Linear  | 33    
2 | rnn           | GRU     | 3.5 K 
3 | dropout       | Dropout | 0     
------------------------------------------
6.7 K     Trainable params
0         Non-trainable params
6.7 K     Total params
0.027     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
/home/gcl/RA/jonahk/miniconda3/envs/DS-discovery/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.4611295759677887     │
│      total_test_loss      │    0.4611295759677887     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.4611295759677887, 'total_test_loss': 0.4611295759677887}]